In [2]:
# Data Manipulation and Handling
import polars as pl
import pandas as pd
import numpy as np
import psycopg2

# DB Credentials
from dotenv import load_dotenv
import os
import sys
from sqlalchemy import create_engine

# Machine Learning Libraries
import torch
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Handling Imbalanced Data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

# Gradient Boosting Libraries
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Model Lifecycle Management
import mlflow
import mlflow.sklearn

# Distributed Computing
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier as SparkRFClassifier

# Model Interpretability
import shap

# Hyperparameter Optimization
import optuna

# Automated Feature Engineering
import featuretools as ft

# Add parent directory to sys.path
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Custom Modules
from fetch_data_hook import fetch_sql_code, fetch_sql_file

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.


## Step 1: Load the Data

In [8]:
equity_data = fetch_sql_code('select * from equity_value_data')
equity_data

,timestamp,close_equity,user_id
0,2017-08-16,60.1000,c44bff59a9debd1da8002a45f24de808
1,2017-08-17,59.8900,c44bff59a9debd1da8002a45f24de808
2,2017-08-18,59.9600,c44bff59a9debd1da8002a45f24de808
3,2016-12-08,504.2000,02ba8e7b2c5996ec667597d6a36e8b01
4,2016-12-09,507.0000,02ba8e7b2c5996ec667597d6a36e8b01
...,...,...,...
1119153,2017-08-09,60.4100,c44bff59a9debd1da8002a45f24de808
1119154,2017-08-10,59.9600,c44bff59a9debd1da8002a45f24de808
1119155,2017-08-11,59.9900,c44bff59a9debd1da8002a45f24de808
1119156,2017-08-14,60.5098,c44bff59a9debd1da8002a45f24de808


In [9]:
features_data = fetch_sql_code('select * from features_data')
features_data

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id
0,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,1970-01-01,stock,40.0,med_time_horizon,895044c23edc821881e87da749c01034
1,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,1970-01-01,stock,200.0,short_time_horizon,458b1d95441ced242949deefe8e4b638
2,med_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,1970-01-01,stock,25.0,long_time_horizon,c7936f653d293479e034865db9bb932f
3,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,1970-01-01,stock,100.0,short_time_horizon,b255d4bd6c9ba194d3a350b3e76c6393
4,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,1970-01-01,stock,20.0,long_time_horizon,4a168225e89375b8de605cbc0977ae91
...,...,...,...,...,...,...,...,...,...
5579,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,1970-01-01,stock,300.0,long_time_horizon,03880c726d8a4e5db006afe4119ad974
5580,med_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,1970-01-01,stock,100.0,short_time_horizon,ae8315109657f44852b24c6bca4decd6
5581,med_risk_tolerance,no_investment_exp,very_important_liq_need,both,1970-01-01,stock,50.0,short_time_horizon,f29c174989f9737058fe808fcf264135
5582,med_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,1970-01-01,stock,100.0,long_time_horizon,24843497d1de88b2e7233f694436cb3a


## Step 2: Generate the Complete Calendar

In [17]:
# Get the min and max dates from the equity data
min_date = equity_data['timestamp'].min()
max_date = equity_data['timestamp'].max()

# Create a date range of all calendar dates
all_dates = pd.date_range(start=min_date, end=max_date, freq='D')
all_dates

DatetimeIndex(['2016-08-16', '2016-08-17', '2016-08-18', '2016-08-19',
               '2016-08-20', '2016-08-21', '2016-08-22', '2016-08-23',
               '2016-08-24', '2016-08-25',
               ...
               '2017-08-09', '2017-08-10', '2017-08-11', '2017-08-12',
               '2017-08-13', '2017-08-14', '2017-08-15', '2017-08-16',
               '2017-08-17', '2017-08-18'],
              dtype='datetime64[ns]', length=368, freq='D')

## Step 3: Identify Market Open and Closed Days

In [18]:
# Determine market open days (dates with any data)
market_open_days =set(equity_data['timestamp']) #equity_data['timestamp'].dt.date.unique()

# Create a DataFrame for all dates with market status
calendar_df = pd.DataFrame({'date': all_dates})
calendar_df['market_status'] = calendar_df['date'].apply(lambda x: 'open' if x in market_open_days else 'closed') #calendar_df['date'].dt.date.apply(lambda x: 'open' if x in market_open_days else 'closed')


## Step 4: Prepare User-Date Data

In [19]:
# Get the list of all users
user_ids = equity_data['user_id'].unique()

# Create a MultiIndex with all users and all dates
# user_date_index = pd.MultiIndex.from_product(
#     [user_ids, all_dates], names=['user_id', 'date']
# )
# Create a DataFrame with this index
# user_date_df = pd.DataFrame(index=user_date_index).reset_index()

# Convert user_ids and all_dates into DataFrames
user_df = pd.DataFrame({'user_id': user_ids})
date_df = pd.DataFrame({'date': all_dates})

# Perform a cross join between user_df and date_df
user_date_df = user_df.merge(date_df, how='cross')

# Merge with the calendar DataFrame to get market status
user_date_df = user_date_df.merge(calendar_df, on='date', how='left')

In [20]:
user_date_df

,user_id,date,market_status
0,bcef4fa9b0bdf22bcf7deae708decf03,2016-08-16,open
1,bcef4fa9b0bdf22bcf7deae708decf03,2016-08-17,open
2,bcef4fa9b0bdf22bcf7deae708decf03,2016-08-18,open
3,bcef4fa9b0bdf22bcf7deae708decf03,2016-08-19,open
4,bcef4fa9b0bdf22bcf7deae708decf03,2016-08-20,closed
...,...,...,...
2054907,98580360c4fb5b0ec511cd87f0d429ed,2017-08-14,open
2054908,98580360c4fb5b0ec511cd87f0d429ed,2017-08-15,open
2054909,98580360c4fb5b0ec511cd87f0d429ed,2017-08-16,open
2054910,98580360c4fb5b0ec511cd87f0d429ed,2017-08-17,open


## Step 5: Merge Equity Data

In [21]:
# Prepare equity data by setting the index
equity_data['date'] = equity_data['timestamp'].dt.date
equity_data['date'] = pd.to_datetime(equity_data['date'])

equity_data = equity_data[['user_id', 'date', 'close_equity']]

# Merge user-date DataFrame with equity data
user_date_df = user_date_df.merge(
    equity_data, on=['user_id', 'date'], how='left'
)


## Step 6: Determine Equity State

In [22]:
# Determine the equity state
def determine_state(row):
    if pd.notnull(row['close_equity']):
        return 'equity_ge_10'
    elif row['market_status'] == 'open':
        return 'equity_lt_10'
    else:
        return None  # Will fill this later

user_date_df['state'] = user_date_df.apply(determine_state, axis=1)


## Step 7: Carry Forward the Last Known State

In [23]:
# Sort the DataFrame
user_date_df = user_date_df.sort_values(['user_id', 'date'])

# Group by user and fill forward the state
user_date_df['state'] = user_date_df.groupby('user_id')['state'].ffill()

# For any remaining missing states at the beginning, fill with 'equity_ge_10'
user_date_df['state'] = user_date_df['state'].fillna('equity_ge_10')


## Step 8: Identify Consecutive Periods of Equity < $10

In [24]:
# # Final Update
# Flag days where equity is less than $10
user_date_df['below_10'] = user_date_df['state'] == 'equity_lt_10'

# Ensure the DataFrame is sorted
user_date_df = user_date_df.sort_values(['user_id', 'date']).reset_index(drop=True)

# Identify consecutive sequences without using apply()
user_date_df['grp'] = (
    (user_date_df['below_10'] != user_date_df.groupby('user_id')['below_10'].shift())
    .fillna(1)  # Handle NaN values resulting from the shift
    .astype(int)
    .groupby(user_date_df['user_id'])
    .cumsum()
)

# Filter only the periods where equity was below $10
below_10_df = user_date_df[user_date_df['below_10']]



## Step 9: Calculate Sequence Lengths

In [25]:
# Calculate the length of each sequence
sequence_lengths = (
    below_10_df.groupby(['user_id', 'grp'])
    .agg(start_date=('date', 'min'), end_date=('date', 'max'), num_days=('date', 'count'))
    .reset_index()
)

# Calculate the number of calendar days in each sequence
sequence_lengths['num_calendar_days'] = (
    sequence_lengths['end_date'] - sequence_lengths['start_date']
).dt.days + 1  # Add 1 to include both start and end dates


## Step 10: Identify Users Who Have Churned

In [26]:
# Identify sequences where num_calendar_days >= 28
churned_sequences = sequence_lengths[sequence_lengths['num_calendar_days'] >= 28]

# Get the list of churned users
churned_users = churned_sequences['user_id'].unique()

## Step 11: Calculate the Percentage of Users Who Have Churned

In [27]:
total_users = len(user_ids)
num_churned_users = len(churned_users)
percentage_churned = (num_churned_users / total_users) * 100

print(f"Percentage of users who have churned: {percentage_churned:.2f}%")

Percentage of users who have churned: 45.38%
